In [1]:
import rasterio
from tqdm import tqdm
from shapely.geometry import Point, box
import pandas as pd
import geopandas as gpd
from rasterio.transform import rowcol
import numpy as np
import os

The function was developed based on the same funtion in https://github.com/stevenpawley/Pyspatialml

In [2]:
def extract_to_point(b, rc):
    """
    Extract the value for the points
    """
    extracted_values = [b[coord[0], coord[1]] for coord in rc]
    return extracted_values

In [10]:
def patch_samples(raster, patch_size, number_patches,strata=False):
    
    raster = rasterio.open(raster)
    Px_SIZE = raster.transform[0]
    xmin, ymin, xmax, ymax = raster.bounds
    PATCH_Px = patch_size
    NUM_PATCHES = number_patches
    PATCH_S = PATCH_Px * Px_SIZE
    
    if strata is False:
        X = np.random.rand(NUM_PATCHES) * ((xmax-(PATCH_S) ) - (xmin + (PATCH_S))) + xmin
        Y = np.random.rand(NUM_PATCHES) * ((ymax - (PATCH_S)) - (ymin + (PATCH_S))) + ymin
        PATCH_CENTER = gpd.GeoSeries.from_xy(X,Y)
        rowcol_tuple = np.transpose(raster.index(PATCH_CENTER.x, PATCH_CENTER.y))

        values = extract_to_point(raster.read()[0,...], rowcol_tuple)
        PATCH_GEOM = PATCH_CENTER.apply(lambda p: box(p.x - PATCH_S/2, p.y - PATCH_S/2, p.x + PATCH_S/2, p.y + PATCH_S/2))

        gdf = gpd.GeoDataFrame({'value':values, 'geometry':PATCH_GEOM},crs=raster.crs)
        gdf = gdf[gdf.value > 0].reset_index(drop=True)
    
    else:
        
        raster_arr = raster.read(masked=True)
        categories = np.unique(raster_arr.flatten())
        categories = categories[~categories.mask]
        selected = np.zeros((0, 2)).astype("int")
        
        for cat in categories:

            # get row,col positions for cat strata
            ind = np.transpose(np.nonzero(raster_arr == cat))

            if NUM_PATCHES > ind.shape[0]:
                msg = (
                    "Sample size is greater than number of pixels in " "strata {}"
                ).format(str(ind))

                msg = os.linesep.join([msg, "Sampling using replacement"])
                Warning(msg)

            # random sample
            samples = np.random.uniform(0, ind.shape[0], NUM_PATCHES).astype("int")
            xy = ind[samples, 1:]
            
            selected = np.append(selected, xy, axis=0)
            rowcol_idx = np.column_stack((selected[:,0], selected[:,1]))
            
            
            
        values = extract_to_point(raster.read()[0,...], rowcol_idx)
        
        X,Y = rasterio.transform.xy(transform=raster.transform, rows=selected[:, 0], cols=selected[:, 1])
        PATCH_CENTER = gpd.GeoSeries.from_xy(X,Y)
        PATCH_GEOM = PATCH_CENTER.apply(lambda p: box(p.x - PATCH_S/2, p.y - PATCH_S/2, p.x + PATCH_S/2, p.y + PATCH_S/2))
        
        gdf = gpd.GeoDataFrame({'value':values, 'geometry':PATCH_GEOM},crs=raster.crs)
    
    return gdf

In [11]:
%%time
gdf = patch_samples('../data/ground_truth/terrain_truth_epsg32654_A.tif', 10, 500, strata=False)

CPU times: user 298 ms, sys: 50.9 ms, total: 349 ms
Wall time: 346 ms


In [12]:
gdf.head()

,value,geometry
0,4,"POLYGON ((329740.930 3928496.414, 329740.930 3..."
1,4,"POLYGON ((323659.597 3934938.183, 323659.597 3..."
2,4,"POLYGON ((341336.375 3923210.152, 341336.375 3..."
3,4,"POLYGON ((312713.519 3955970.930, 312713.519 3..."
4,4,"POLYGON ((312314.477 3948955.089, 312314.477 3..."


In [13]:
%%time
gdf = patch_samples('../data/ground_truth/terrain_truth_epsg32654_A.tif', 16, 500, strata=True)

CPU times: user 7.06 s, sys: 1.53 s, total: 8.59 s
Wall time: 8.59 s


In [14]:
gdf.head()

,value,geometry
0,0,"POLYGON ((366365.000 3945565.000, 366365.000 3..."
1,0,"POLYGON ((339705.000 3932985.000, 339705.000 3..."
2,0,"POLYGON ((361035.000 3978985.000, 361035.000 3..."
3,0,"POLYGON ((398295.000 3986355.000, 398295.000 3..."
4,0,"POLYGON ((376425.000 3963745.000, 376425.000 3..."
